## Imports sqlite and connects to GBQ database

In [3]:
import sqlite3

In [4]:
#connect to GBQ instance

# Do our imports for the code
import random
from google.cloud import bigquery
from google.oauth2 import service_account


# These first two values will be different on your machine. 
service_path = "C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\"
service_file = 'ada-Wedge-b7fc47715ad0.json' # this is your authentication information  
gbq_proj_id = 'ada-wedge'  # change this to your project_id
gbq_dataset_id = 'wedge2' # and change this to your data set ID

private_key =service_path + service_file

# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
#for reference (so I know which types the fields are)

cur.execute('''DROP TABLE IF EXISTS transactions''')
cur.execute('''CREATE TABLE transactions ( 
    datetime TIMESTAMP,
    register_no INTEGER,
    emp_no INTEGER,
    trans_no INTEGER,
    upc INTEGER,
    description TEXT,
    trans_type TEXT,
    trans_subtype TEXT,
    trans_status TEXT,
    department INTEGER,
    quantity REAL,
    Scale INTEGER,
    cost REAL,
    unitPrice REAL,
    total REAL,
    regPrice REAL,
    altPrice REAL,
    tax INTEGER,
    taxexempt INTEGER,
    foodstamp INTEGER,
    wicable INTEGER,
    discount REAL,
    memDiscount REAL,
    discountable INTEGER,
    discounttype INTEGER,
    voided INTEGER,
    percentDiscount REAL,
    ItemQtty REAL,
    volDiscType INTEGER,
    volume INTEGER,
    VolSpecial REAL,
    mixMatch INTEGER,
    matched INTEGER,
    memType INTEGER,
    staff INTEGER,
    numflag INTEGER,
    itemstatus INTEGER,
    tenderstatus INTEGER,
    charflag TEXT,
    varflag INTEGER,
    batchHeaderID INTEGER,
    local INTEGER,
    organic INTEGER,
    display INTEGER,
    receipt INTEGER,
    card_no INTEGER,
    store INTEGER,
    branch INTEGER,
    match_id INTEGER,
    trans_id INTEGER)''')

NameError: name 'cur' is not defined

## Table 1 --1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .


In [9]:
##pulls in the correct data for table 1 using the below query


query = ("""

SELECT
(EXTRACT(date FROM datetime)) AS date,
(EXTRACT(hour FROM datetime)) AS hour,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS Transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
      ELSE 1
    END ) AS Items
FROM
  `ada-wedge.wedge2.transArch*`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
group BY
 date,
 hour
 
order by
date,
hour

    
""")


    
    

#runs the query job

query_job = client.query(
    query,
    location="US",
)




table1 = client.query(query) 





In [10]:
#turns query results to a string and writes to a text file seperating results by a comma
with open('query1.txt','a') as output_file:
    for line in table1:
        row = [str(item) for item in line]
        output_file.write(",".join(row)+'\n')
        

In [11]:
##renames the file saved for the 1st query, connects to database
input_file = 'query1.txt'
db = sqlite3.connect("WEDGETables.db")
cur = db.cursor()
   
#creates a table called table1 with the following headers
cur.execute('''DROP TABLE IF EXISTS Table1''')
cur.execute('''CREATE TABLE Table1 (
    date TIMESTAMP,
    hour TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

#opens the file, runs through each line in the file and writes it to the table
with open(input_file,'r',encoding="Latin-1") as infile :
    for idx, line in enumerate(infile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO Table1 (date,hour,sales,transactions,items)
            VALUES (?,?,?,?,?)''', line)
    
db.commit() # commits the changes to the table in DB


## Table 2 --Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.


In [12]:
#pulls in the correct data for table 2 using the below query


query = ("""

SELECT
  card_no,
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
  SUM(total) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS Transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS Items
FROM
  `ada-wedge.wedge2.transArchi*`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  card_no,
  year,
  month
Order by 
card_no,
year,
month
    
""")


    
    

#runs the query job

query_job = client.query(
    query,
    location="US",
)




table2 = client.query(query) 





In [13]:
#turns query results to a string and writes to a text file seperating results by a comma
with open('query2.txt','a') as output_file:
    for line in table2:
        row = [str(item) for item in line]
        output_file.write(",".join(row)+'\n')
        

In [14]:
#renames the file saved for the 2nd query, connects to database
input_file2 = 'query2.txt'
db = sqlite3.connect("WEDGETables.db")
cur = db.cursor()

#creates a table called table2 with the following headers
cur.execute('''DROP TABLE IF EXISTS Table2''')
cur.execute('''CREATE TABLE Table2 (
    card_no INTEGER,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

#opens the file, runs through each line in the file and writes it to the table
with open(input_file2,'r',encoding="Latin-1") as infile :
    for idx, line in enumerate(infile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO Table2 (card_no,year,month,sales,transactions,items)
            VALUES (?,?,?,?,?,?)''', line)
    
db.commit()  # commits the changes to the table in DB

## TABLE 3:Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.


In [15]:
##pulls in the correct data for table 3 using the below query


query = ("""

SELECT `ada-wedge.wedge2.department_lookup`.dept_name as department_name, upc, description, `ada-wedge.wedge2.transArchive*`.department as department_number,
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
SUM(total) AS sales,
COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS Transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS Items 
FROM
`ada-wedge.wedge2.department_lookup` JOIN `ada-wedge.wedge2.transArchive*` ON
`ada-wedge.wedge2.department_lookup`.department = `ada-wedge.wedge2.transArchive*`.department 
  
  
WHERE
  card_no != 3
  AND `ada-wedge.wedge2.transArchive*`.department != 0 --need to specify which table for department
  AND `ada-wedge.wedge2.transArchive*`.department != 15 --need to specify what table for department
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
   

group by
year,
month,
department_name,
upc,
description,
department_number

order by
description,
year, 
month
 

    
""")


    
    

#runs the query job

query_job = client.query(
    query,
    location="US",
)




table3 = client.query(query) 





In [16]:
#turns query results to a string and writes to a text file seperating results by a comma
with open('query3.txt','a') as output_file:
    for line in table3:
        row = [str(item) for item in line]
        output_file.write(",".join(row)+'\n')
        
        

In [17]:
##renames the file saved for the 3rd query, connects to database
input_file3 = 'query3.txt'
db = sqlite3.connect("WEDGETables.db")
cur = db.cursor()

#creates a table called table3 with the following headers
    
cur.execute('''DROP TABLE IF EXISTS Table3''')
cur.execute('''CREATE TABLE Table3 (
    department_name TEXT,
    UPC INTEGER,
    description TEXT,
    department_number INTEGER,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

#opens the file, runs through each line in the file and writes it to the table

with open(input_file3,'r',encoding="Latin-1") as infile :
    for idx, line in enumerate(infile.readlines()) :
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO Table3 (department_name,UPC,description,department_number,year,month,sales,transactions,items)
            VALUES (?,?,?,?,?,?,?,?,?)''', line)
    
db.commit() # commits the changes to the table in DB